In [ ]:
%pip install --user --quiet ciw
import ciw

In [ ]:
from dataclasses import dataclass

@dataclass
class ModelVariant:
    num_servers: int
    queue_capacity: int
    arrival_dist: ciw.dists.Distribution
    service_dist: ciw.dists.Distribution

@dataclass
class SimResult:
    variant: ModelVariant
    task_count: int
    utilization: float
    loss_probability: float
    mean_wait_time: float
    mean_residence_time: float
    
    def __init__(self, variant: ModelVariant, sim: ciw.Simulation):
        self.variant = variant
        self.utilization = sim.transitive_nodes[0].server_utilisation
        
        tasks = sim.get_all_records()
        self.task_count = len(tasks)
        self.loss_probability = len(sim.rejection_dict[1][0]) / self.task_count
        self.mean_wait_time = sum(t.waiting_time for t in tasks) / self.task_count
        self.mean_residence_time = sum(t.waiting_time + t.service_time for t in tasks) / self.task_count
        
def simulate(v: ModelVariant, task_count: int) -> SimResult:
    m = ciw.create_network(
        arrival_distributions=[v.arrival_dist],
        service_distributions=[v.service_dist],
        number_of_servers=[v.num_servers],
        queue_capacities=[v.queue_capacity]
    )
    s = ciw.Simulation(m)
    s.simulate_until_max_customers(task_count, method='Finish')
    return SimResult(v, s)

simulate(ModelVariant(
    num_servers=2,
    queue_capacity=1,
    arrival_dist = ciw.dists.Exponential(1 / 20),
    service_dist = ciw.dists.Exponential(1 / 21)
), 1000)